In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits import mplot3d
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA

%matplotlib inline
pd.set_option("display.max_columns", None)

# Lab 23 - Dimension Reduction and Principal Components Analysis

*Principal Components Analysis* or *PCA* is a method for reducing the dimension of the data.  The dimension of the data is the number of columns or variables in the data.  We encounter dimension reduction in every day life every time we look at a 2D picture or photo of a 3D object.  Some ways of reducing the dimension are more useful than others.  For example, if someone takes a photo looking straight down at a table, it might be much harder to tell that it is a table than if the photo is taken at an angle so you can see the table top and all 4 table legs.  

Principal Components Analysis tries to map the data to lower dimensions in such a way that as much variation as possible is retained.  Specifically, PCA reduces the dimensions of the data set by successively finding the directions with the most variation and using these directions as the new coordinate system.

See [https://setosa.io/ev/principal-component-analysis/](https://setosa.io/ev/principal-component-analysis/) for a visual, interactive explaination of PCA.

### Section 1: Loading and cleaning the penguin data

We will start by looking at the Palmer penguin dataset from Labs 3, 21, and 22.  First load the data into a variable.

Data URL: https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv

Drop the rows with missing data.

For technical reasons later in the lab, we need to reset the index as well.  Recall the function is `reset_index(drop = True)`.

<details><summary>Answer:</summary>
<code>
penguins = penguins.reset_index(drop = True)
</code>
</details>

We can use the qualitative variables `species`, `island`, and `sex` as well as the quantitative variables to do PCA by first turning them into dummy variables.

Because PCA looks for the direction with the highest variance, we need to pre-process our data so that each column has the same variance.  Otherwise a column with high variance will dominate.  The scaler we have used in previous labs does not make sure the variances are the same.

Instead, we will *standardize* each column of data, by taking the *z-score* of each data point.  After standardizing a column of data, the mean of that column will be 0 and the variance and standard deviation will be 1.

To do this, we create a StandardScaler object.

In [ ]:
standardizer = StandardScaler()

And then use it to transform our data.  Notice the function is the same as for scaling, just applied to a StandardScaler object instead of a MinMaxScaler object.

In [ ]:
penguins2_standardized = standardizer.fit_transform(penguins2)
penguins2_standardized

### Section 2: Principal Components Analysis

Let's find the first two principal components for our processed penguin dataset (which we assume is stored in the variable `penguins2_standardized`):

In [ ]:
# create the PCA object
pca = PCA(n_components=2)

# find the first two principal components of the penguin data
pca.fit(penguins2_standardized)

# find the coordinates of the penguin data in the new coordinate system
penguins_pca = pca.transform(penguins2_standardized)

Display `penguins_pca`.  Is it a DataFrame?

Let's put the data from `penguins_pca` (which is a numpy array) into a DataFrame, along with the species names.

In [ ]:
# put the coordinates into a DataFrame
penguins_pca_df = pd.DataFrame(penguins_pca, columns = ["PC1","PC2"])
# add the species names as a new column
penguins_pca_df["species"] = penguins["species"]

Display this dataframe.

Use Seaborn to plot a scatter plot where x is PC1 and y is PC2, with the points colored by the species.

What do you notice about this plot? What variation is shown along PC1?  Along PC2?

Let's compare this scatter plot with the scatter plots for all pairs of quantitative variables, colored by the species.

How do these scatterplots compare to the PCA one?  Which shows the most separation between species?  Which species is most separated from the others in the scatterplots of the original quantitiative variables?  How is this reflected in the scatterplot of the PCA coordinates?

### Section 3:  Cleaning and pre-processing the labor market data

Let's apply PCA to the labor market data from Homeworks 21 and 22.

Recall the Federal Reserve Bank of New York has information about the labor market for recent college graduates [here](https://www.newyorkfed.org/research/college-labor-market/college-labor-market_compare-majors.html).

Data URL: [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/labor-market-Feb2021.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/labor-market-Feb2021.csv)

Load the data, skipping the first 13 rows and the last 3 rows.

Remove the commas in the columns `Median Wage Early Career` and `Median Wage Mid-Career`, and then change the type of the columns to `int`.

Create a new DataFrame `x` without the `Major` column, since it is qualitative.  Since each major is different, we can't easily convert this column into quantitative data using dummy variables.

Finally, create a new variable `x_standardized` with the standardized columns of `x`.  That is, with the data in each column of `x` transformed so that it has mean 0 and variance 1 (and hence also standard deviation 1).

You can use the same `standardizer` variable from Section 1.

### Section 4: Applying PCA to the labor market data

Find the first two principal components using the standardized labor data.

Create a DataFrame with the two new coordinates, as we did in Section 2.

Plot these new coordinates.  We are not coloring them yet, as it is not clear what to color them with.

What do you notice about the plot?  Do you see any distinct clusters?

Let's use k-means clustering to cluster the standardized labor data.

First use either the elbow method or the silhouette score to determine the number of clusers.

<details><summary>Answer:</summary>
(for silhouette score method)
<code>
silhouette_score_list = []
for k in range(2,11):
    kmeans = KMeans(n_clusters = k)
    kmeans_clusters = kmeans.fit_predict(x_standardized)
    score = silhouette_score(x_standardized,kmeans_clusters)
    silhouette_score_list.append(score)
plt.plot(range(2,11), silhouette_score_list)
</code>
</details>

Using the k you just determined, cluster the standardized labor data. k-means to cluster the scaled labor data.

Add the cluster labels to your DataFrame with the PCA coordinates.

Plot the colored-by-cluster data using the principal component coordinates.

How are the clusters divided relative to the new PCA coordinates?  